- 在深度学习模型日益庞大的今天，并非所有人都能满足从头开始训练一个模型的软硬件条件，稀缺的数据和昂贵的计算资源都是我们需要面对的难题。迁移学习可以帮助我们缓解在数据和计算资源上的尴尬。作为当前深度学习领域中最重要的方法论之一，迁移学习有着自己自身的理论依据和实际效果验证。
- 作为一门实验性学科，深度学习通常需要反复的实验和结果论证。在现在和将来，是否有海量的数据资源和强大的计算资源，这是决定学界和业界深度学习和人工智能发展的关键因素。通常情况下，获取海量的数据资源对于企业而言并非易事，尤其是对于像医疗等特定领域，要想做一个基于深度学习的医学影像的辅助诊断系统，大量且高质量的打标数据非常关键。但通常而言，不要说高质量，就是想获取大量的医疗数据就已困难重重。
- 那怎么办呢？是不是获取不了海量的数据研究就一定进行不下去了？当然不是。因为我们有迁移学习。那究竟什么是迁移学习？顾名思义，迁移学习就是利用数据、任务或模型之间的相似性，将在旧的领域学习过或训练好的模型，应用于新的领域这样的一个过程。从这段定义里面，我们可以窥见迁移学习的关键点所在，即新的任务与旧的任务在数据、任务和模型之间的相似性。
- 本节我们介绍迁移学习中的一种常用技术：微调（fine tuning）。微调由以下4步构成。
1. 在源数据集（如ImageNet数据集）上预训练一个神经网络模型，即源模型。
2. 创建一个新的神经网络模型，即目标模型。它复制了源模型上除了输出层外的所有模型设计及其参数。我们假设这些模型参数包含了源数据集上学习到的知识，且这些知识同样适用于目标数据集。我们还假设源模型的输出层跟源数据集的标签紧密相关，因此在目标模型中不予采用。
3. 为目标模型添加一个输出大小为目标数据集类别个数的输出层，并随机初始化该层的模型参数。
4. 在目标数据集（如FashionMNIST数据集）上训练目标模型。我们将从头训练输出层，而其余层的参数都是基于源模型的参数微调得到的。
<center/><img src="https://cdn.nlark.com/yuque/0/2021/jpeg/1508544/1614151636690-a8025370-aa39-48e5-a34e-e2fad2816cde.jpeg"/></center>
- 当目标数据集远小于源数据集时，微调有助于提升模型的泛化能力。
# 加载数据集
- FashionMNIST是28×28的灰度图片，60000/10000的训练测试数据划分，其涵盖了来自10种类别的共7万个不同商品的正面图片。
<center/><img src="https://cdn.nlark.com/yuque/0/2021/png/1508544/1614153298387-a8d9fcaa-119a-4b0a-ac5e-b5f18478e077.png"/></center>
- 假设我们想从图像中识别出不同种类的衣物。一种可能的方法是先收集尽可能多的衣物的不同拍摄角度的图片，然后在收集到的图像数据集上训练一个分类模型，但样本数仍然不及ImageNet数据集中样本数的十分之一。这可能会导致适用于ImageNet数据集的复杂模型在这个椅子数据集上过拟合。同时，因为数据量有限，最终训练得到的模型的精度也可能达不到实用的要求。
- 为了应对上述问题，一个显而易见的解决办法是收集更多的数据。然而，收集和标注数据会花费大量的时间和资金。例如，为了收集ImageNet数据集，研究人员花费了数百万美元的研究经费。虽然目前的数据采集成本已降低了不少，但其成本仍然不可忽略。
- 另外一种解决办法是应用迁移学习（transfer learning），将从源数据集学到的知识迁移到目标数据集上。例如，虽然ImageNet数据集的图像大多跟衣物无关，但在该数据集上训练的模型可以抽取较通用的图像特征，从而能够帮助识别边缘、纹理、形状和物体组成等。这些类似的特征对于识别衣物也可能同样有效。

In [ ]:
# 注意：fashion mnist数据集加载，需要先安装ipywidgets，不然会报错提示需要升级jupyter
!pip install ipywidgets --user

In [1]:
# 在CPU版本自动下载数据
import time
import torch
import numpy as np
from torch import nn, optim
import torchvision

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

###################fashion mnist数据集加载######################
def load_data_fashion_mnist(batch_size, resize=None, root='./Datasets/FashionMNIST'):
    """Download the fashion mnist dataset and then load into memory."""
    trans = []
    if resize:
        trans.append(torchvision.transforms.Resize(size=resize))
    trans.append(torchvision.transforms.ToTensor())
    
    transform = torchvision.transforms.Compose(trans)
    mnist_train = torchvision.datasets.FashionMNIST(root=root, train=True, download=True, transform=transform)
    mnist_test = torchvision.datasets.FashionMNIST(root=root, train=False, download=True, transform=transform)
    train_iter = torch.utils.data.DataLoader(mnist_train, batch_size=batch_size, shuffle=True)
    test_iter = torch.utils.data.DataLoader(mnist_test, batch_size=batch_size, shuffle=False)

    return train_iter, test_iter
#################################################################
batch_size = 64
train_iter, test_iter = load_data_fashion_mnist(batch_size, resize=7)

/home/admin/.local/lib/python3.6/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


Extracting ./Datasets/FashionMNIST/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./Datasets/FashionMNIST/FashionMNIST/raw


Extracting ./Datasets/FashionMNIST/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./Datasets/FashionMNIST/FashionMNIST/raw


Extracting ./Datasets/FashionMNIST/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./Datasets/FashionMNIST/FashionMNIST/raw


Extracting ./Datasets/FashionMNIST/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./Datasets/FashionMNIST/FashionMNIST/raw
Processing...


/home/admin/.local/lib/python3.6/site-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Done!


# 模型使用
Pytorchvision支持多种图像分类模型，这里我们选择残差网络模型作为迁移学习的基础模型，对输出层（最后一层）改为十个类别，其它特征层选择在训练时候微调参数。常见的ResNet网络模型如下：
<img src="https://cdn.nlark.com/yuque/0/2021/png/1508544/1614152398516-530cf2a8-0eb7-4277-887f-c701e2b24011.png"/>
基于ResNet18完成网络模型修改，最终的模型实现代码如下：

In [2]:
# 在CPU版本自动下载模型参数
class SurfaceDefectResNet(torch.nn.Module):
    def __init__(self):
        super(SurfaceDefectResNet, self).__init__()
        # Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /home/admin/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth
        self.cnn_layers = torchvision.models.resnet18(pretrained=True)
        num_ftrs = self.cnn_layers.fc.in_features
        self.cnn_layers.fc = torch.nn.Linear(num_ftrs, 10)

    def forward(self, x):
        # stack convolution layers
        out = self.cnn_layers(x)
        return out

net = SurfaceDefectResNet()

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /home/admin/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth


# 定义损失函数

In [3]:
loss = nn.CrossEntropyLoss()

# 定义优化函数

In [4]:
optimizer = torch.optim.SGD(net.parameters(), lr=0.1)

# 定义准确率

In [5]:
def accuracy(y_hat, y):
    return (y_hat.argmax(dim=1) == y).float().mean().item()

def evaluate_accuracy(data_iter, net):
    acc_sum, n = 0.0, 0
    for X, y in data_iter:
        acc_sum += (net(X).argmax(dim=1) == y).float().sum().item()
        n += y.shape[0]
    return acc_sum / n

# 训练

In [ ]:
num_epochs = 1

def train_ch(net, train_iter, test_iter, loss, num_epochs, batch_size,
              params=None, lr=None, optimizer=None):
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n = 0.0, 0.0, 0
        for X, y in train_iter:
            X = torch.cat((X, X, X), 1)
            y_hat = net(X)
            l = loss(y_hat, y).sum()
            
            # 梯度清零
            if optimizer is not None:
                optimizer.zero_grad()
            elif params is not None and params[0].grad is not None:
                for param in params:
                    param.grad.data.zero_()
            
            l.backward()
            optimizer.step() 
            
            train_l_sum += l.item()
            train_acc_sum += (y_hat.argmax(dim=1) == y).sum().item()
            n += y.shape[0]
        test_acc = evaluate_accuracy(test_iter, net)
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f'
              % (epoch + 1, train_l_sum / n, train_acc_sum / n, test_acc))

train_ch(net, train_iter, test_iter, loss, num_epochs, batch_size, None, None, optimizer)

# 练习题
选择题：
1. 假设我们将源模型的输出层改成输出大小为目标数据集类别个数的输出层，则对于这个新的输出层如何初始化  
    a. 复制源模型的参数进行初始化  
    b. 随机初始化参数  
    c. 用全零初始化参数  
    d. 不需要初始化  
2. 假设我们将源模型的输出层改成输出大小为目标数据集类别个数的输出层，在训练过程中下列说法正确的是  
    a. 对输出层使用较大的学习率，对其他层使用较小的学习率。  
    b. 对输出层使用较小的学习率，对其他层使用较大的学习率。  
    c. 对输出层和其他层使用相同大小的学习率。    
    d. 对输出层进行微调，其他层保持参数不变，不需要学习。  

答案：
1. b
2. a